In [2]:
#Descomentar esto para istalar las librerias
# !pip install requests
# !pip install lxml
# !pip install selenium
# !pip install unidecode
# !pip install beautifulsoup4

from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from unidecode import unidecode
import requests
import lxml as lxml
import html as html
import time
import pandas as pd
import numpy as np 
import re

In [3]:
#Link a la pagina web
link = 'https://diaonline.supermercadosdia.com.ar/'
link_sin_barra = 'https://diaonline.supermercadosdia.com.ar'

In [4]:
# Inicializa el driver de Selenium (asegúrate de tener el driver correspondiente instalado)
driver = webdriver.Chrome()  # Puedes cambiar a otro navegador y/o ruta del driver si lo necesitas
driver.maximize_window()

#Abre la página web en el navegador controlado por Selenium
driver.get(link)

#Hacemos click en el boton de categorias
button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'diaio-custom-mega-menu-0-x-custom-mega-menu-trigger__button')))
button.click()

# Obtiene el HTML después de cargar todos los elementos
html_home = driver.page_source

# Cierra el navegador controlado por Selenium
driver.quit()

#Se busca dentro del HTML las categorias donde hacer hover
sp_home = BeautifulSoup(html_home, 'lxml')
sp_categorias = sp_home.find("div",{"class":"diaio-custom-mega-menu-0-x-category-list__container"}).find_all("a")
categorias = []
for cat in sp_categorias:
    # href = cat.get("href")
    categorias.append(cat.get("href"))
categorias 

['https://diaonline.supermercadosdia.com.ar/almacen',
 'https://diaonline.supermercadosdia.com.ar/bebidas',
 'https://diaonline.supermercadosdia.com.ar/frescos',
 'https://diaonline.supermercadosdia.com.ar/desayuno',
 'https://diaonline.supermercadosdia.com.ar/limpieza',
 'https://diaonline.supermercadosdia.com.ar/perfumeria',
 'https://diaonline.supermercadosdia.com.ar/congelados',
 'https://diaonline.supermercadosdia.com.ar/bebes-y-ninos',
 'https://diaonline.supermercadosdia.com.ar/hogar-y-deco',
 'https://diaonline.supermercadosdia.com.ar/mascotas',
 'https://diaonline.supermercadosdia.com.ar/almacen/golosinas-y-alfajores?PS=15&O=',
 'https://diaonline.supermercadosdia.com.ar/frescos/frutas-y-verduras',
 'https://diaonline.supermercadosdia.com.ar/electro-hogar']

In [ ]:
#Primer categoria
driver = webdriver.Chrome()  # Puedes cambiar a otro navegador y/o ruta del driver si lo necesitas
driver.maximize_window()
# Abre la página web en el navegador controlado por Selenium
driver.get(categorias[0])
html_categoria = driver.page_source

#Se obtiene la lista de categorias internas 
sp_categoria_interna = BeautifulSoup(html_categoria, 'lxml')
categorias_internas = sp_categoria_interna.find("div", {"class":"diaio-search-result-0-x-filter__container diaio-search-result-0-x-filter__container--plp bb b--muted-4 diaio-search-result-0-x-filter__container--category-2"})
lista_checkboxes = categorias_internas.find("div", {"class":"diaio-search-result-0-x-filterTemplateOverflow diaio-search-result-0-x-filterTemplateOverflow--plp pb5 overflow-y-auto"}).find_all("input", {"type":"checkbox"})

#Apretar boton pop-up
button_popup = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, "vtex-modal__close-icon")))
button_popup.click()

#Lista de categorias dentro de la pagina
id_chechbox = []
for l in lista_checkboxes:
    id_chechbox.append(l.get("id"))
id_chechbox


#Obtencion del HTML que contiene todos los productos de una categoria
productos_obtenidos_html = {}

for i in id_chechbox:
    checkbox = driver.find_element(By.ID, i)
    checkbox.click()  # Activar el checkbox
    time.sleep(2)

    # Iniciar la altura inicial para el bucle
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Desplazarse un poco hacia abajo
        driver.execute_script("window.scrollBy(0, 1500);")
        time.sleep(2)  # Esperar para cargar más contenido

        # Obtener la nueva altura de la página
        new_height = driver.execute_script("return document.body.scrollHeight")

        # Intentar hacer clic en el botón "Mostrar más"
        try:
            see_more_button = driver.find_element(By.XPATH, "//button[.//div[text()='Mostrar más']]")
            if see_more_button.is_displayed():
                # Desplazar el botón a la vista
                driver.execute_script("arguments[0].scrollIntoView({ behavior: 'smooth', block: 'center' });", see_more_button)
                time.sleep(1)
                # Verificar si está habilitado y hacer clic
                if see_more_button.is_enabled():
                    see_more_button.click()
                    print("Botón apretado")
                    time.sleep(2)  # Esperar después del clic
        except Exception as e:
            # Capturar el error pero seguir el bucle
            print(f"No se pudo hacer clic en el botón")

        # Salir del bucle si no hay más contenido
        if new_height == last_height:
            break

        # Actualizar la altura de la página para la siguiente iteración
        last_height = new_height

    # Regresar al inicio de la página
    time.sleep(1)
    driver.execute_script("window.scrollTo(0, 0);")

    # Obtener el HTML de los productos
    html_productos = driver.page_source
    sp_productos = BeautifulSoup(html_productos, 'lxml')
    productos_obtenidos_html[i] = sp_productos 
    time.sleep(2)

    # Deseleccionar el checkbox antes de pasar a la siguiente categoría
    checkbox.click()
    time.sleep(1)

driver.quit()


Botón apretado
Botón apretado
Botón apretado
Botón apretado
Botón apretado
Botón apretado
Botón apretado
No se pudo hacer clic en el botón: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[.//div[text()='Mostrar más']]"}
  (Session info: chrome=131.0.6778.109); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF761E46CF5+28821]
	(No symbol) [0x00007FF761DB3880]
	(No symbol) [0x00007FF761C5578A]
	(No symbol) [0x00007FF761CA91BE]
	(No symbol) [0x00007FF761CA94AC]
	(No symbol) [0x00007FF761CF2647]
	(No symbol) [0x00007FF761CCF33F]
	(No symbol) [0x00007FF761CEF412]
	(No symbol) [0x00007FF761CCF0A3]
	(No symbol) [0x00007FF761C9A778]
	(No symbol) [0x00007FF761C9B8E1]
	GetHandleVerifier [0x00007FF76217FCED+3408013]
	GetHandleVerifier [0x00007FF76219745F+3504127]
	GetHandleVerifier [0x00007FF76218B63D+3455453]
	GetHandl

In [ ]:
#Primer categoria
driver = webdriver.Chrome()  # Puedes cambiar a otro navegador y/o ruta del driver si lo necesitas
driver.maximize_window()
primer_url = link + categorias[0]


# Abre la página web en el navegador controlado por Selenium
driver.get(primer_url)
button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "vtex-search-result-3-x-seeMoreButton")))
time.sleep(3)
driver.execute_script("arguments[0].click();", button)
time.sleep(3)

# Obtiene el HTML después de cargar todos los elementos
html_2 = driver.page_source

# Cierra el navegador controlado por Selenium
# Parsea el HTML con BeautifulSoup
soup_2 = BeautifulSoup(html_2, 'lxml')
containers = soup_2.find_all('div', class_='vtex-search-result-3-x-filter__container')
containers


#creamos el filtro que nos devuelve las sub-categorias
filtros = []
sub_categorias_sin_procesar = []
sub_categorias = []

for i in range(len(containers)):
    value = containers[i].get_text()
    filtros.append(value)
    test = re.findall('Sub-Categoría', filtros[i])
    if test:
        texto_lista = containers[i].get_text(separator='\n', strip=True).split('\n')
        sub_categorias_sin_procesar.extend(texto_lista[1:-1]) #Exluimos el ultimo registro

#limpiamos el texto para poder generar las variables que necesitamos
for i in sub_categorias_sin_procesar:
    texto_procesado = unidecode(i).lower()
    texto = texto_procesado.replace(' ','-')
    texto_2 = texto.replace(',', '')
    sub_categorias.append(texto_2)

driver.quit()

for s in range(len(sub_categorias)):
    primer_url = link + categorias[0]
    driver = webdriver.Chrome()  # Puedes cambiar a otro navegador y/o ruta del driver si lo necesitas
    driver.maximize_window()
    driver.get(primer_url)
    button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "vtex-search-result-3-x-seeMoreButton")))
    time.sleep(3)
    driver.execute_script("arguments[0].click();", button)
    time.sleep(3)
    selector_sub_categoria = f'label[for="category-3-{sub_categorias[s]}"]' #Creamos la categoria que necesitamos para hacer click
    checkbox = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, selector_sub_categoria)))
    checkbox.click()
    time.sleep(5)
    
    prev_elements_count = len(driver.find_elements(By.CLASS_NAME, 'vtex-search-result-3-x-galleryItem'))

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        curr_elements_count = len(driver.find_elements(By.CLASS_NAME, 'vtex-search-result-3-x-galleryItem'))

        if curr_elements_count == prev_elements_count:
            break
        
        # Actualizar el contador de elementos
        prev_elements_count = curr_elements_count

    # Obtiene el HTML después de cargar todos los elementos
    html_3 = driver.page_source
    time.sleep(2)

    # Cierra el navegador controlado por Selenium
    driver.quit()

    soup_3 = BeautifulSoup(html_3, 'lxml')
    productos_categoria = soup_3.find_all('div', class_='vtex-search-result-3-x-galleryItem')
    productos_categoria

    total_items = []

    for litem_listado in range(len(productos_categoria)):
        try:
            # Extraemos link de la imagen del producto
            imagen = productos_categoria[litem_listado].find('img')
            link_imagen = imagen['src'] if imagen else None

            # Extraemos link a la web del producto en el supermercado
            web_producto = productos_categoria[litem_listado].find('a')
            url_producto = link_sin_barra + web_producto['href'] if web_producto else None

            # Datos del producto
            informacion_producto_wrapped = productos_categoria[litem_listado].find('div', 'vtex-flex-layout-0-x-flexCol')
            informacion_producto = informacion_producto_wrapped.find_all('div', 'vtex-flex-layout-0-x-flexColChild') if informacion_producto_wrapped else []

            # Creamos las listas soporte
            datos_productos_sin_procesar = []
            datos_productos_procesados = []
            datos_productos_procesados.append(categorias[0])  # Agregamos la categoria a la que pertenece el producto
            datos_productos_procesados.append(sub_categorias[s])  # Agregamos la categoria a la que pertenece el producto

            # Con este bucle for extraemos la informacion sin procesar
            for info in informacion_producto:
                valor_lista = info.get_text(strip=True).replace('\xa0', ' ')
                datos_productos_sin_procesar.append(valor_lista)

            # Generamos la expresion regular que extrae el precio correcto
            # y conformamos la lista con los datos del producto
            for i in range(0, 4):
                datos_productos_procesados.append(datos_productos_sin_procesar[i] if i < len(datos_productos_sin_procesar) else None)

            for i in range(len(datos_productos_sin_procesar)):
                patron_regex_precios = r"(\$\d{1,3}(?:\.\d{3})*,\d{2})"
                resultado_regex = re.findall(patron_regex_precios, datos_productos_sin_procesar[i])
                if resultado_regex:
                    datos_productos_procesados.extend(resultado_regex)

            if len(datos_productos_procesados) == 7:
                datos_productos_procesados.append(None)
                datos_productos_procesados.append(datos_productos_sin_procesar[-1])
            else:
                datos_productos_procesados.append(datos_productos_sin_procesar[-1])

            datos_productos_procesados.append(url_producto)  # Agregamos el link al producto
            datos_productos_procesados.append(link_imagen)  # Agregamos el link a la imagen

            total_items.append(datos_productos_procesados)
        except Exception as e:
            print(f"Error procesando el producto {litem_listado}: {e}")

    nombre_dataframe = f'data_producto_{categorias[0]}_{sub_categorias[s]}'
    lista_columnas = ('categoria', 'producto', 'banner_producto', 'marca', 'descripcion_producto',
                    'promocion', 'precio', 'precio_sin_descuento', 'precio_por_kilo',
                    'url_producto', 'url_imagen')

    globals()[nombre_dataframe] = pd.DataFrame(total_items, columns=lista_columnas)
    ruta_csv = f'C:/Users/phoen/OneDrive/Escritorio/Lauta/Data Science/Projects/scrapper_dia/Productos obtenidos/{nombre_dataframe}.csv'
    globals()[nombre_dataframe].to_csv(ruta_csv, index=False)

## Next steps

Poder profesionalizar este entorno para que cualquiera pueda acceder
